# Story Idea Generation Exploration

This notebook explores different approaches to generating story ideas using LLMs.

**Approaches covered:**
1. **Priming** - Have the model recall famous stories to enter "fiction writer" mode
2. **Random Word Sampling** - Sample words from tokenizer vocabulary as creative seeds
3. **Structured Random Elements** - Use LLM-driven randomness for story parameters
4. **Constraint Mashup** - Combine incompatible constraints for creative tension

## Setup

In [ ]:
# Install dependencies if needed
# !pip install openai tiktoken nltk

In [1]:
import random
import json
from openai import OpenAI

# Load API key
with open("credential", "r") as f:
    OPENROUTER_API_KEY = f.read().strip()

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

# Model to use
MODEL = "anthropic/claude-sonnet-4" # Using Sonnet for exploration (cheaper), switch to opus for final
MODEL_OPUS = "anthropic/claude-opus-4"

def call_llm(messages, model=MODEL, temperature=0.8, max_tokens=2000):
    """Call the LLM via OpenRouter."""
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

print("Setup complete!")

Setup complete!


---
## Approach 0: Priming with Famous Stories

Before generating ideas, we prime the model by having it recall famous stories. This activates the "fiction writer" context in the model's latent space.

In [23]:
# Load genre list from genres.py
from genres import ALL_GENRES, DOCUMENT_TYPES

print(f"Loaded {len(ALL_GENRES)} genres (pre-1970)")
print(f"Loaded {len(DOCUMENT_TYPES)} document types")
print(f"\nSample genres: {random.sample(ALL_GENRES, 10)}")

Loaded 114 genres (pre-1970)
Loaded 56 document types

Sample genres: ['locked room mystery', 'regency romance', 'legal thriller', 'hard science fiction', 'picaresque', 'contemporary fiction', 'haunted house', 'essay', 'hardboiled', 'noir']


In [57]:
random.randrange(1900, 1991, 10)

1960

In [59]:
def prime_with_multi_genre_stories(n_genres=5):
    """
    Sample n random genres and ask the model to recall famous stories from each.
    This primes the model with diverse literary traditions before generation.
    """
    # Sample random genres
    sampled_genres = random.sample(ALL_GENRES, n_genres)
    
    # Randomly select a time period
    start_year = random.randrange(1910, 1971, 10)
    end_year = start_year + 50
    time_period = f"{start_year}-{end_year}"
    
    genre_list = "\n".join(f"- {genre}" for genre in sampled_genres)
    
    priming_prompt = f"""Before we begin generating story ideas, let's warm up your creative circuits.

Here are 5 randomly selected literary genres:
{genre_list}

For each genre, recall ONE famous story (published roughly around {time_period}) that exemplifies it. 
For each story, explain in 1-2 sentences what narrative technique or structural choice made it memorable.
Focus on specific craft elements: how did the author create tension, develop character, or subvert expectations?"""
    
    messages = [
        {"role": "system", "content": "You are a literary analyst with deep knowledge of storytelling across genres and eras."},
        {"role": "user", "content": priming_prompt}
    ]
    
    result = call_llm(messages, temperature=0.7)
    
    return {
        "genres": sampled_genres,
        "time_period": time_period,
        "priming_text": result
    }

# Test multi-genre priming
multi_genre_prime = prime_with_multi_genre_stories()
print(f"Sampled genres: {multi_genre_prime['genres']}")
print(f"Time period: {multi_genre_prime['time_period']}")
print(f"\n{'='*50}\n")
print(multi_genre_prime['priming_text'])

Sampled genres: ['epic fantasy', 'spy fiction', 'whodunit', 'noir', 'historical mystery']
Time period: 1960-2010


Here are five exemplary stories from that era and their key craft elements:

**Epic Fantasy: *The Lord of the Rings* (1954-1955) by J.R.R. Tolkien**
Tolkien masterfully employed interwoven parallel narratives that gradually converged, creating mounting tension as readers tracked multiple storylines of increasing urgency. His technique of withholding Frodo and Sam's journey for entire chapters while following other characters created anxiety about their fate and made their eventual scenes feel more precious and hard-won.

**Spy Fiction: *Tinker Tailor Soldier Spy* (1974) by John le Carré**
Le Carré revolutionized the genre by structuring the novel as a methodical investigation rather than an action thriller, with George Smiley slowly peeling back layers of deception through quiet observation and psychological insight. The author created tension through information asymmetry

In [ ]:
def prime_with_famous_stories(genre=None):
    """Have the model recall famous stories to prime it for fiction generation."""
    
    # Randomly select a 5-year period between 1900 and 2020
    start_year = random.randint(1900, 1990)  # 2015 so we can add 5 years
    end_year = start_year + 30
    time_period = f"{start_year}-{end_year}"
    
    genre_clause = f" in the {genre} genre" if genre else ""
    
    priming_prompt = f"""Before we begin generating story ideas, let's warm up your creative circuits.

Think of 3 famous stories{genre_clause} published between {time_period}. 
For each one, identify what made it memorable. What narrative trick, thematic resonance, or structural choice made each story work?"""
    
    messages = [
        {"role": "system", "content": "You are a literary analyst with knowledge of storytelling."},
        {"role": "user", "content": priming_prompt}
    ]
    
    return call_llm(messages, temperature=0.7)

# Test priming
priming_result = prime_with_famous_stories()
print(priming_result)

Here are three enduring stories and what makes them unforgettable:

## "The Gift of the Magi" by O. Henry

**What makes it work:** The perfect ironic reversal. Both characters sacrifice their most precious possession to buy a gift that complements the other's treasure—rendering both gifts "useless" yet infinitely meaningful. This isn't just a clever twist; it's a structural mirror that reflects the story's theme. The narrative trick *becomes* the meaning: true love lies in the sacrifice itself, not the practical outcome.

## "Cinderella" (various versions)

**What makes it work:** The transformation fantasy taps into a universal psychological need—the belief that our true worth will eventually be recognized. But the story's genius lies in its specific imagery: glass slippers, pumpkin carriages, midnight deadlines. These concrete details make the impossible feel tangible. The story also follows a perfect rhythm of oppression → magic → triumph → test → reward that mirrors how we hope jus

In [ ]:
# Genre-specific priming
priming_scifi = prime_with_famous_stories(genre="science fiction")
print(priming_scifi)

---
## Approach 1: Random Word Sampling (Baseline)

Sample 20 random words from the NLTK English dictionary (~235k words), then ask the LLM to generate a story idea from them.

Unlike BPE tokenizers (tiktoken, etc.) which contain subword fragments, NLTK's words corpus contains complete English words.

---
## Approach 1: Random Word Sampling (Baseline)

Sample 20 random words from the GPT-4 tokenizer vocabulary, **validated against NLTK dictionary**.

This hybrid approach gives us:
- **From tokenizer**: Words that are common enough to appear in modern text (not archaic/obscure)
- **From dictionary**: Validation that these are complete real words (not subword fragments like `'geom'` or `'compreh'`)

In [85]:
import tiktoken
import nltk

# Download NLTK words corpus for dictionary validation
nltk.download('words', quiet=True)
from nltk.corpus import words as nltk_words

# Build a set of valid English words for fast lookup
english_dict = set(w.lower() for w in nltk_words.words())
print(f"Dictionary size: {len(english_dict)} words")

# Load GPT-4 tokenizer (cl100k_base encoding)
enc = tiktoken.get_encoding("cl100k_base")

def get_real_words_from_tokenizer(encoding, max_id=110000, min_len=4, max_len=15):
    """
    Extract tokens from LLM tokenizer, but only keep those that are 
    real English words (validated against dictionary).
    
    This gives us words that are:
    1. Common enough to be in an LLM's vocabulary (actually used in text)
    2. Complete real words (not subword fragments)
    """
    real_words = []
    for i in range(max_id):
        try:
            token = encoding.decode([i])
            # Tokens with leading space are word boundaries
            if token.startswith(" "):
                word = token.strip().lower()
                # Check: alphabetic, right length, AND in dictionary
                if (word.isalpha() and 
                    min_len <= len(word) <= max_len and
                    word in english_dict):
                    real_words.append(word)
        except:
            continue
    return list(set(real_words))

vocab_words = get_real_words_from_tokenizer(enc, max_id=75000)
print(f"Extracted {len(vocab_words)} real words from tokenizer (validated against dictionary)")
print(f"Sample: {random.sample(vocab_words, 20)}")

Dictionary size: 234377 words
Extracted 10491 real words from tokenizer (validated against dictionary)
Sample: ['funny', 'inner', 'eligible', 'melee', 'underscore', 'edge', 'capitol', 'paar', 'chain', 'saint', 'ethiopia', 'coupling', 'divers', 'shout', 'exchange', 'phill', 'compost', 'alberto', 'popcorn', 'protection']


In [86]:
import re

def sample_random_words(n=20):
    """Sample n random words from vocabulary."""
    return random.sample(vocab_words, min(n, len(vocab_words)))


def generate_idea_from_words(words, priming_context=None):
    """Generate a story idea using the given words as creative seeds."""
    
    word_list = ", ".join(words)
    
    prompt = f"""Here are 20 randomly selected words:

{word_list}

Using at least 5 of these words as inspiration (not necessarily literally), generate a compelling story idea (3-4 sentences). 
Before you generate, think about: 
What is the core situation? What makes this story impossible to put down? 
What is the primary form this story should take? (such as personal letter, journal, interview transcript, bureaucratic report, diplomatic correspondence, research log, notebook, company memo, obituary, field notes, pamphlet, ad, telegram, etc.)
Your response should include the story idea only. No intro, no outro. Be specific. Avoid generic tropes."""
    
    messages = []
    
    # Include priming context if provided
    if priming_context:
        messages.append({"role": "system", "content": "You are a creative writing expert who just analyzed what makes great stories work."})
        messages.append({"role": "assistant", "content": priming_context})
    else:
        messages.append({"role": "system", "content": "You are a creative writing expert skilled at finding unexpected story seeds."})
    
    messages.append({"role": "user", "content": prompt})
    
    return call_llm(messages, temperature=0.9)

# Test: Generate without priming
words = sample_random_words(20)
print(f"Random words: {words}\n")
print("=" * 50)
idea_no_prime = generate_idea_from_words(words)
print(idea_no_prime)

Random words: ['nathan', 'screened', 'oracle', 'delaware', 'prosperous', 'welfare', 'conditioned', 'tour', 'aggressively', 'browse', 'petite', 'evil', 'myself', 'achievement', 'salon', 'shel', 'stated', 'former', 'commander', 'always']



KeyboardInterrupt: 

In [73]:
words = sample_random_words(20)
print(f"Random words: {words}\n")
print("=" * 50)
idea_no_prime = generate_idea_from_words(words, priming_context=multi_genre_prime['priming_text'])
print(idea_no_prime)

Random words: ['sitting', 'strange', 'upcoming', 'chat', 'alex', 'voter', 'introduce', 'colon', 'mutable', 'adapt', 'rencontre', 'protest', 'covid', 'ship', 'harassment', 'moscow', 'armed', 'find', 'note', 'vital']

**Moscow Cultural Exchange Program - Incident Report #447-B**

During a routine diplomatic chat to introduce visiting American voter registration specialist Alex Morrison to our Moscow counterparts, Morrison discovered a handwritten note in Cyrillic tucked inside a library book about pandemic protocols. The note detailed a strange rencontre (meeting) between Russian officials and an armed protest organizer, suggesting systematic harassment of opposition voters through deliberately spreading COVID misinformation in targeted districts. When Morrison attempted to photograph the evidence, the library's security system mysteriously malfunctioned, and now both Morrison and the Russian librarian who helped translate the note have gone missing, leaving only Morrison's partially enc

## The result looks okay, but there are high-level echoes. 

---
## Approach 2: Structured Random Elements with LLM-Driven Randomness

Instead of selecting from predetermined lists, we use the LLM itself to generate random elements with true creative variation.

In [ ]:
def generate_random_element(element_type, constraints=None):
    """
    Use LLM to generate a random element with true creative variation.
    The key insight: we ask for MULTIPLE options with specific variety requirements,
    then randomly select one. This gives us LLM creativity + true randomness.
    """
    
    constraint_text = f"\nAdditional constraint: {constraints}" if constraints else ""
    
    prompts = {
        "time_period": f"""Generate 7 wildly different time periods for a story setting.
Include:
- At least one prehistoric or ancient
- At least one that's a very specific year (not just "the 1920s" but "1923")
- At least one far future
- At least one that's oddly specific ("the week after the 1906 San Francisco earthquake")
- Mix real historical periods with speculative ones{constraint_text}

Return ONLY a JSON array of strings, nothing else.""",
        
        "location": f"""Generate 7 wildly different locations for a story setting.
Include:
- At least one that's a specific real place (not just "a city" but "the basement of the Prague astronomical clock tower")
- At least one that's an unusual institutional setting (hospital morgue, patent office, lighthouse)
- At least one natural environment with a twist
- At least one that's in transit (ship, train, caravan)
- Mix mundane and extraordinary{constraint_text}

Return ONLY a JSON array of strings, nothing else.""",
        
        "document_type": f"""Generate 7 unusual document formats a story could take.
Go beyond the obvious (letter, diary). Consider:
- Official documents (incident report, autopsy, insurance claim, court transcript)
- Institutional records (personnel file, surveillance log, maintenance checklist)
- Personal artifacts (shopping list, margin notes in a book, voicemail transcript)
- Technical documents (field observation notes, equipment manual, recipe)
- Communication records (diplomatic cable, intercepted message, complaint form){constraint_text}

Return ONLY a JSON array of strings, nothing else.""",
        
        "narrative_constraint": f"""Generate 7 unusual narrative constraints that could make a story interesting.
Examples of the KIND of thing (don't use these exactly):
- The narrator doesn't realize they're dead
- Every paragraph must be exactly 50 words
- The story is told in reverse chronological order
- The narrator is an unreliable AI
Be creative and specific.{constraint_text}

Return ONLY a JSON array of strings, nothing else."""
    }
    
    messages = [
        {"role": "system", "content": "You are a creative writing randomizer. Return ONLY valid JSON arrays."},
        {"role": "user", "content": prompts[element_type]}
    ]
    
    response = call_llm(messages, temperature=1.0, max_tokens=500)
    
    # Parse JSON response
    try:
        # Handle potential markdown code blocks
        if "```" in response:
            response = response.split("```")[1]
            if response.startswith("json"):
                response = response[4:]
        options = json.loads(response.strip())
        # Randomly select one
        return random.choice(options)
    except json.JSONDecodeError:
        print(f"Failed to parse JSON: {response}")
        return response  # Return raw if parsing fails

# Test each element type
print("Time period:", generate_random_element("time_period"))
print("Location:", generate_random_element("location"))
print("Document type:", generate_random_element("document_type"))
print("Narrative constraint:", generate_random_element("narrative_constraint"))

In [ ]:
def generate_structured_story_idea(include_priming=True):
    """
    Generate a story idea using structured random elements.
    Each element is generated with LLM creativity + random selection.
    """
    
    # Generate random elements
    print("Generating random elements...")
    time_period = generate_random_element("time_period")
    location = generate_random_element("location")
    document_type = generate_random_element("document_type")
    narrative_constraint = generate_random_element("narrative_constraint")
    
    print(f"\nSelected elements:")
    print(f"  Time: {time_period}")
    print(f"  Location: {location}")
    print(f"  Document: {document_type}")
    print(f"  Constraint: {narrative_constraint}")
    print("\n" + "=" * 50 + "\n")
    
    # Build the generation prompt
    prompt = f"""Create a story idea using these specific parameters:

**Time Period:** {time_period}
**Location:** {location}
**Document Format:** {document_type}
**Narrative Constraint:** {narrative_constraint}

Generate a compelling story concept that:
1. Naturally incorporates ALL four elements
2. Has a clear central tension or mystery
3. Features a specific, interesting narrator voice
4. Could only exist in this exact combination of parameters

Provide:
- **Premise** (3-4 sentences)
- **Opening line** (the actual first line of the document)
- **The hidden truth** (what the reader will discover that the narrator may not fully understand)
"""
    
    messages = []
    
    if include_priming:
        messages.append({"role": "system", "content": "You are a master of found-document fiction, skilled at creating authentic-feeling artifacts that reveal larger stories."})
        # Add a brief priming
        messages.append({"role": "user", "content": "Before we begin, what makes found-document fiction (epistolary novels, fake archives) so effective? One sentence."})
        messages.append({"role": "assistant", "content": "Found-document fiction works because the gap between what the document says and what actually happened creates a space where the reader becomes an active detective, piecing together the truth from fragments the narrator never intended to reveal."})
    else:
        messages.append({"role": "system", "content": "You are a creative writing expert."})
    
    messages.append({"role": "user", "content": prompt})
    
    return call_llm(messages, temperature=0.85)

# Generate a structured story idea
structured_idea = generate_structured_story_idea()
print(structured_idea)

In [ ]:
# Generate multiple ideas to see variety
print("Generating 3 different structured story ideas...\n")
for i in range(3):
    print(f"\n{'='*60}")
    print(f"IDEA {i+1}")
    print(f"{'='*60}\n")
    idea = generate_structured_story_idea(include_priming=True)
    print(idea)

---
## Approach 3: Constraint Mashup

Generate story ideas by combining deliberately incompatible or surprising constraint pairs. The creative tension comes from resolving contradictions.

In [ ]:
def generate_constraint_pair():
    """Generate a pair of constraints that seem incompatible."""
    
    prompt = """Generate 5 pairs of story constraints that seem incompatible or contradictory at first glance.

Examples of what I mean:
- "A horror story" + "told entirely through wholesome family recipes"
- "An epic spanning centuries" + "that takes place in real-time during a 3-minute phone call"
- "A love story" + "where the lovers never meet"

The pairs should create CREATIVE TENSION - the story becomes interesting because the writer must resolve the apparent contradiction.

Return as JSON array of objects with "constraint_a" and "constraint_b" keys."""
    
    messages = [
        {"role": "system", "content": "You are a creative writing constraint generator. Return ONLY valid JSON."},
        {"role": "user", "content": prompt}
    ]
    
    response = call_llm(messages, temperature=1.0, max_tokens=800)
    
    try:
        if "```" in response:
            response = response.split("```")[1]
            if response.startswith("json"):
                response = response[4:]
        pairs = json.loads(response.strip())
        return random.choice(pairs)
    except:
        print(f"Parse error: {response}")
        return {"constraint_a": "a mystery", "constraint_b": "told in footnotes only"}

# Test
pair = generate_constraint_pair()
print(f"Constraint A: {pair['constraint_a']}")
print(f"Constraint B: {pair['constraint_b']}")

In [ ]:
def generate_mashup_idea():
    """Generate a story idea from incompatible constraints."""
    
    pair = generate_constraint_pair()
    
    print(f"Constraints to reconcile:")
    print(f"  A: {pair['constraint_a']}")
    print(f"  B: {pair['constraint_b']}")
    print("\n" + "="*50 + "\n")
    
    prompt = f"""You must create a story idea that satisfies BOTH of these seemingly incompatible constraints:

Constraint A: {pair['constraint_a']}
Constraint B: {pair['constraint_b']}

Your job is to find the elegant solution - the story concept where both constraints aren't just accommodated but actually STRENGTHEN each other.

Provide:
1. **The Resolution**: How do these constraints actually work together? (2-3 sentences)
2. **The Story**: Premise, setting, central character (3-4 sentences)
3. **Why It Works**: What does this combination reveal that neither constraint alone would? (1-2 sentences)
"""
    
    messages = [
        {"role": "system", "content": "You are a master of constrained writing, finding creative solutions where others see impossibility."},
        {"role": "user", "content": prompt}
    ]
    
    return call_llm(messages, temperature=0.85)

# Generate mashup idea
mashup_idea = generate_mashup_idea()
print(mashup_idea)

---
## Approach 4: Full Pipeline - Priming + Random + Structure

Combine all approaches into a comprehensive story idea generator.

In [ ]:
def full_pipeline_generate(use_opus=False):
    """
    Full pipeline combining all approaches:
    1. Prime with famous stories
    2. Generate random word seeds
    3. Generate structured elements
    4. Synthesize into final idea
    """
    
    model = MODEL_OPUS if use_opus else MODEL
    print(f"Using model: {model}\n")
    
    # Step 1: Prime
    print("Step 1: Priming with famous stories...")
    priming = prime_with_famous_stories()
    print("[Priming complete]\n")
    
    # Step 2: Random words
    print("Step 2: Sampling random words...")
    words = sample_random_words(10)  # Fewer words, more focused
    print(f"Words: {words}\n")
    
    # Step 3: Structured elements
    print("Step 3: Generating structured elements...")
    time_period = generate_random_element("time_period")
    location = generate_random_element("location")
    document_type = generate_random_element("document_type")
    
    print(f"  Time: {time_period}")
    print(f"  Location: {location}")
    print(f"  Document: {document_type}\n")
    
    # Step 4: Synthesize
    print("Step 4: Synthesizing final idea...")
    print("="*60 + "\n")
    
    synthesis_prompt = f"""You are generating a story idea. You have been given:

**Random word seeds (use at least 3 as thematic inspiration):**
{', '.join(words)}

**Structural parameters:**
- Time period: {time_period}
- Location: {location}  
- Document format: {document_type}

Create a story idea that:
1. Weaves in themes suggested by the random words
2. Is set specifically in the given time and place
3. Takes the form of the specified document type
4. Has a compelling mystery or tension at its core
5. Features a narrator whose voice feels authentic to the document type

Provide:
- **Logline** (one sentence pitch)
- **Setup** (the situation, 3-4 sentences)
- **The Document** (what is this artifact and why does it exist?)
- **The Narrator** (who wrote this and what do they want?)
- **The Hidden Layer** (what truth lurks beneath the surface?)
- **Opening Lines** (the first 2-3 sentences of the actual story)
"""
    
    messages = [
        {"role": "system", "content": "You are a master storyteller who has just been analyzing what makes great fiction work."},
        {"role": "assistant", "content": priming},  # Include priming as context
        {"role": "user", "content": synthesis_prompt}
    ]
    
    result = call_llm(messages, model=model, temperature=0.85, max_tokens=1500)
    return result

# Run full pipeline
final_idea = full_pipeline_generate(use_opus=False)  # Set to True for final quality
print(final_idea)

---
## Comparison: Run All Approaches

Generate one idea from each approach to compare quality and variety.

In [ ]:
def compare_approaches():
    """Run all approaches and display results for comparison."""
    
    results = {}
    
    # Approach 1: Random words only (no priming)
    print("\n" + "="*70)
    print("APPROACH 1: Random Words (Baseline)")
    print("="*70)
    words = sample_random_words(20)
    print(f"Words: {words}\n")
    results['random_words'] = generate_idea_from_words(words, priming_context=None)
    print(results['random_words'])
    
    # Approach 2: Structured elements
    print("\n" + "="*70)
    print("APPROACH 2: Structured Random Elements")
    print("="*70)
    results['structured'] = generate_structured_story_idea(include_priming=True)
    print(results['structured'])
    
    # Approach 3: Constraint mashup
    print("\n" + "="*70)
    print("APPROACH 3: Constraint Mashup")
    print("="*70)
    results['mashup'] = generate_mashup_idea()
    print(results['mashup'])
    
    # Approach 4: Full pipeline
    print("\n" + "="*70)
    print("APPROACH 4: Full Pipeline (All Combined)")
    print("="*70)
    results['full'] = full_pipeline_generate(use_opus=False)
    print(results['full'])
    
    return results

# Run comparison (this will take a few minutes due to multiple API calls)
# comparison_results = compare_approaches()

---
## Using Opus 4.5 for Final Generation

For production-quality ideas, use the full pipeline with Opus.

In [ ]:
# Generate a production-quality idea with Opus 4.5
print("Generating with Claude Opus 4.5...\n")
opus_idea = full_pipeline_generate(use_opus=True)
print(opus_idea)

---
## Utility: Batch Generation for Selection

In [ ]:
def generate_idea_batch(n=5, approach="structured"):
    """Generate multiple ideas for human selection."""
    
    ideas = []
    for i in range(n):
        print(f"\nGenerating idea {i+1}/{n}...")
        
        if approach == "structured":
            idea = generate_structured_story_idea(include_priming=True)
        elif approach == "mashup":
            idea = generate_mashup_idea()
        elif approach == "random_words":
            words = sample_random_words(20)
            idea = f"Words: {words}\n\n" + generate_idea_from_words(words)
        else:
            idea = full_pipeline_generate()
            
        ideas.append(idea)
        print(f"\n{'='*60}")
        print(f"IDEA {i+1}")
        print(f"{'='*60}")
        print(idea)
    
    return ideas

# Generate a batch of ideas (uncomment to run)
# batch = generate_idea_batch(n=3, approach="structured")

---
## Summary

| Approach | Pros | Cons | Best For |
|----------|------|------|----------|
| **Random Words** | Maximum unpredictability, forces novel connections | Can produce incoherent results | Breaking creative blocks |
| **Structured Elements** | Coherent, specific settings | May feel formulaic | Found-document fiction |
| **Constraint Mashup** | Creative tension drives innovation | Requires more synthesis effort | Experimental fiction |
| **Full Pipeline** | Combines all benefits, highest quality | Slower, more API calls | Production story generation |

### Recommendations

1. **For exploration**: Use structured elements with `document_type` focus
2. **For breaking blocks**: Use random word sampling
3. **For final generation**: Use full pipeline with Opus 4.5
4. **Key insight**: Priming with famous stories consistently improves output quality